In [174]:
import sys
import datetime as dt
from dateutil.relativedelta import *
import time
import numpy as np  # array operations
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
import pandas as pd 
import calendar
import concurrent.futures
import matplotlib as mpl
import matplotlib.pyplot as plt  # standard plotting library
from pylab import plt
plt.style.use('ggplot')
# put all plots in the notebook itself
"""
  Local  Class
"""
import CAPM

In [175]:
def my_Read_csv(stocklist) :
    global  DataPath
    #  read csv from yahoo 
    data = pd.DataFrame()
    for sym in stocklist:
        data[sym] = pd.read_csv(DataPath+"/"+sym+".csv", index_col=0)['Adj Close']
        if data[sym].dtype !=  float :
            data = data[data[sym] != 'null']
    data = data.astype(float)
    return data

In [176]:
"""
Global variables
"""
DataPath = "Data"  

In [177]:
def Import_Data():
    HSI = ['0027.HK','0700.HK','0005.HK','0939.HK','1299.HK','0941.HK','1398.HK','3988.HK','0001.HK','2318.HK','0388.HK',
           '0386.HK','0002.HK','0883.HK','0016.HK','0823.HK','2388.HK','0003.HK','0011.HK','0857.HK','2628.HK',
                 '0006.HK','0688.HK','1928.HK','0004.HK','0267.HK','0175.HK','0762.HK','0066.HK','1088.HK','1109.HK','2018.HK',
                 '0012.HK','0017.HK','3328.HK','0023.HK','1038.HK','2319.HK','0083.HK','0101.HK','1044.HK','0019.HK',
                 '0992.HK','0151.HK','0836.HK','0144.HK','0135.HK','0293.HK','3968.HK','1113.HK','0322.HK']

    stocklist = HSI
    data = my_Read_csv(stocklist)
    return data


In [178]:
def Rebalance(data):
    global DrawChart
    
    aCapm = CAPM.CAPM(data)
    prets, pvols = aCapm.random_walk()
    Sflag, msrstat, msrpw = aCapm.Max_Sharpe()
    Mflag, mvstat, mvpw = aCapm.Min_Varian()
    retH = msrstat[0] * 1.1
    retL = mvstat[0] * 0.7
    trets, tvols = aCapm.EF_Frontier(retH, retL)
    SHRW = pd.DataFrame(index=data.columns)
    SHRW['weight'] = msrpw.round(3)
    MVW = pd.DataFrame(index=data.columns)
    MVW['weight'] = mvpw.round(3)
        
    plt.figure(figsize=(10, 6))
    plt.scatter(pvols, prets,
                    c=prets / pvols, marker='.')
                    # random portfolio composition
    plt.scatter(tvols, trets,
                    c=trets / tvols, 
                    marker='X')
                    # efficient frontier
    plt.plot(msrstat[1], msrstat[0],
                'r*', markersize=15.0)
                # portfolio with highest Sharpe ratio
    plt.plot(mvstat[1], mvstat[0],
                'y*', markersize=15.0)
                # minimum variance portfolio
    plt.grid(True)
    plt.xlabel('expected volatility')
    plt.ylabel('expected return')
    plt.title('Minimum risk portfolios for given return level (crosses) '+sdt+' - '+edt)
    plt.colorbar(label='Sharpe ratio')
    
    return SHRW, MVW

In [179]:
def print_port_weighting(title, port, weights) :
    print('{} :'.format(title))
    for sym, w in zip(port, weights) :
        if (w > 0.0):
            print('{} : {}'.format(sym, w))

In [180]:
endt = dt.date.today()
stdt = endt - relativedelta(years=1)
sdt = stdt.__str__()
edt = endt.__str__()
print(" Extract Data from {} to {}".format(sdt, edt ))
bigdata = Import_Data()
d1 = bigdata[sdt:edt]
ssdt = d1.index[0]
eedt = d1.index[-1]
print(" Actual Data from {} to {}.".format(ssdt, eedt))

 Extract Data from 2016-11-06 to 2017-11-06
 Actual Data from 2016-11-07 to 2017-11-06.


In [181]:
SRw, MVw = Rebalance(d1)

  Max_Sharpe.noa = 51
  max func sharpe=
     fun: -4.058433784353571
     jac: array([  1.73711777e-03,   9.75131989e-04,   1.00886703e-01,
         1.86382222e+00,   9.13933456e-01,   2.53486186e+00,
         1.36101294e+00,   2.38563848e+00,   1.45654809e+00,
        -1.23721361e-03,   2.71148354e+00,   2.57558626e+00,
         1.04266822e+00,   5.24529338e-01,   6.09030247e-01,
         4.76229548e-01,   4.00583804e-01,   7.41188526e-01,
         6.05583191e-05,   1.99456149e+00,   2.38219255e+00,
         1.92828906e+00,   2.99057001e+00,   2.03988796e+00,
         9.30924356e-01,   2.73654431e+00,   3.39186192e-03,
         1.10318148e+00,   8.27954352e-01,   3.14118528e+00,
         2.74009877e+00,   1.16086006e-03,  -1.37323141e-03,
         3.02799106e-01,   3.19329649e+00,   1.74137652e+00,
         7.35172868e-01,   3.84088576e-01,   1.78387880e+00,
         2.52874595e+00,  -2.10046768e-03,   2.17156708e+00,
         2.36943656e+00,  -2.03394890e-03,   1.14067012e+00,
     

In [187]:
tmp = SRw.sort_values(by=['weight'], ascending=False)
second = tmp.head(4)
print('  all second {}'.format(second))

  all SRw          weight
0011.HK   0.446
0700.HK   0.233
2318.HK   0.232
0322.HK   0.089


In [188]:
d2 = pd.DataFrame()
for col in second.index:
    d2[col] = d1[col]
d2.info()
ssdt = d2.index[0]
eedt = d2.index[-1]
print(" D2 Actual Data from {} to {}.".format(ssdt, eedt))

<class 'pandas.core.frame.DataFrame'>
Index: 247 entries, 2016-11-07 to 2017-11-06
Data columns (total 4 columns):
0011.HK    247 non-null float64
0700.HK    247 non-null float64
2318.HK    247 non-null float64
0322.HK    247 non-null float64
dtypes: float64(4)
memory usage: 8.7+ KB
 D2 Actual Data from 2016-11-07 to 2017-11-06.


In [189]:
d2.head()

,0011.HK,0700.HK,2318.HK,0322.HK
Date,,,,
2016-11-07,134.688370,203.728165,40.194965,8.692739
2016-11-08,134.784668,205.124939,40.537678,8.751941
2016-11-09,133.436813,201.932327,39.313713,8.673005
2016-11-10,136.228775,207.718918,40.097050,8.830875
2016-11-11,137.480377,199.537872,39.803299,8.830875


In [190]:
d2.tail()

,0011.HK,0700.HK,2318.HK,0322.HK
Date,,,,
2017-10-31,184.699997,349.799988,68.500000,12.30
2017-11-01,185.600006,360.000000,70.300003,12.62
2017-11-02,184.300003,361.799988,70.050003,12.70
2017-11-03,184.000000,368.000000,70.699997,12.52
2017-11-06,182.699997,377.000000,70.800003,12.36


In [191]:
SRw, MVw = Rebalance(d2)
SRw

  Max_Sharpe.noa = 4
  max func sharpe=
     fun: -3.821240527613922
     jac: array([-0.00030109,  0.00037274,  0.00010222,  0.0002695 ])
 message: 'Optimization terminated successfully.'
    nfev: 51
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([ 0.44645306,  0.23305567,  0.2318035 ,  0.08868777])

  min func variance=
     fun: 0.011693894589345838
     jac: array([ 0.02335035,  0.0227793 ,  0.02390116,  0.02336255])
 message: 'Optimization terminated successfully.'
    nfev: 49
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([ 0.67209163,  0.06856562,  0.13760425,  0.1217385 ])

--> TVOLS dupl value=


,weight
0011.HK,0.446
0700.HK,0.233
2318.HK,0.232
0322.HK,0.089
